# Introction to Tempo on Kubernetes

This notebook will walk you through an end-to-end example deploying a Tempo pipeline deployed to Kubernetes.

## Prerequisites

  * rclone and conda installed.
  * Run this notebook within the `seldon-examples` conda environment. Details to create this can be found [here]().

## Architecture

We will show two Iris dataset prediction models combined with service orchestration that shows some arbitrary python code to control predictions from the two models.

![architecture](architecture.png)

In [1]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## Train Iris Models

We will train:

  * A sklearn logistic regression model
  * A xgboost model

In [2]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
import joblib
iris = datasets.load_iris()
X = iris.data  # we only take the first two features.
y = iris.target
logreg = LogisticRegression(C=1e5)
logreg.fit(X, y)
logreg.predict_proba(X[0:1])
with open("./artifacts/sklearn/model.joblib","wb") as f:
    joblib.dump(logreg, f)

In [3]:
from xgboost import XGBClassifier
clf = XGBClassifier()
clf.fit(X,y)
clf.save_model("./artifacts/xgboost/model.bst")

[07:42:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/clive/anaconda3/envs/tempo-examples/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


## Defining pipeline

The first step will be to define our custom pipeline.
This pipeline will access 2 models, stored remotely. 

In [4]:
import os
import numpy as np
from typing import Tuple
from tempo.serve.metadata import ModelFramework, KubernetesOptions
from tempo.serve.model import Model
from tempo.seldon.docker import SeldonDockerRuntime
from tempo.kfserving.protocol import KFServingV2Protocol
from tempo.serve.utils import pipeline, predictmethod
from tempo.seldon.k8s import SeldonKubernetesRuntime
from tempo.serve.utils import pipeline

import logging
logging.basicConfig(level=logging.INFO)

SKLEARN_FOLDER = os.getcwd()+"/artifacts/sklearn"
XGBOOST_FOLDER = os.getcwd()+"/artifacts/xgboost"
PIPELINE_ARTIFACTS_FOLDER = os.getcwd()+"/artifacts/classifier"

k8s_options = KubernetesOptions(namespace="production",authSecretName="minio-secret")
k8s_runtime = SeldonKubernetesRuntime(k8s_options=k8s_options)
k8s_runtime_v2 = SeldonKubernetesRuntime(k8s_options=k8s_options, protocol=KFServingV2Protocol())


sklearn_model = Model(
        name="test-iris-sklearn",
        runtime=k8s_runtime,
        platform=ModelFramework.SKLearn,
        local_folder=SKLEARN_FOLDER,
        uri="s3://tempo/basic/sklearn"
)

xgboost_model = Model(
        name="test-iris-xgboost",
        runtime=k8s_runtime,
        platform=ModelFramework.XGBoost,
        local_folder=XGBOOST_FOLDER,
        uri="s3://tempo/basic/xgboost"
)

docker_runtime_v2 = SeldonDockerRuntime(protocol=KFServingV2Protocol())

@pipeline(name="classifier",
          runtime=k8s_runtime_v2,
          uri="s3://tempo/basic/pipeline",
          local_folder=PIPELINE_ARTIFACTS_FOLDER,
          models=[sklearn_model, xgboost_model])
def classifier(payload: np.ndarray) -> Tuple[np.ndarray,str]:
    res1 = sklearn_model(payload)

    if res1[0][0] > 0.5:
        return res1,"sklearn prediction"
    else:
        return xgboost_model(payload),"xgboost prediction"

### Saving artifacts

We provide a conda yaml in out `local_folder` which tempo will use as the runtime environment to save. If this file was not there it would save the current conda environment. One can also provide a named conda environment with `conda_env` in the decorator.

In [5]:
%%writefile artifacts/classifier/conda.yaml
name: tempo
channels:
  - defaults
dependencies:
  - _libgcc_mutex=0.1=main
  - ca-certificates=2021.1.19=h06a4308_0
  - certifi=2020.12.5=py37h06a4308_0
  - ld_impl_linux-64=2.33.1=h53a641e_7
  - libedit=3.1.20191231=h14c3975_1
  - libffi=3.3=he6710b0_2
  - libgcc-ng=9.1.0=hdf63c60_0
  - libstdcxx-ng=9.1.0=hdf63c60_0
  - ncurses=6.2=he6710b0_1
  - openssl=1.1.1j=h27cfd23_0
  - pip=21.0.1=py37h06a4308_0
  - python=3.7.9=h7579374_0
  - readline=8.1=h27cfd23_0
  - setuptools=52.0.0=py37h06a4308_0
  - sqlite=3.33.0=h62c20be_0
  - tk=8.6.10=hbc83047_0
  - wheel=0.36.2=pyhd3eb1b0_0
  - xz=5.2.5=h7b6447c_0
  - zlib=1.2.11=h7b6447c_3
  - pip:
    - mlops-tempo
    - mlserver==0.3.1.dev5
    - mlserver-tempo==0.3.1.dev5

Overwriting artifacts/classifier/conda.yaml


In [6]:
classifier.save(save_env=True)

INFO:root:Saving environment
INFO:tempo:Using found conda.yaml
INFO:tempo:Creating conda env with: conda env create --name tempo-0ddef1b9-e296-4681-83cd-9d06303916da --file /tmp/tmp5npulri2.yml
INFO:tempo:packing conda environment from tempo-0ddef1b9-e296-4681-83cd-9d06303916da


Packing environment at '/home/clive/anaconda3/envs/tempo-0ddef1b9-e296-4681-83cd-9d06303916da' to '/home/clive/work/mlops/fork-tempo/docs/examples/intro_k8s/artifacts/classifier/environment.tar.gz'
[########################################] | 100% Completed | 10.3s


INFO:tempo:Removing conda env with: conda remove --name tempo-0ddef1b9-e296-4681-83cd-9d06303916da --all --yes


## Deploying pipeline to K8s

The next step, will be to deploy our pipeline to Kubernetes.
We will divide this process into 3 sub-steps:

1. Save our artifacts and environment
2. Upload to remote storage
3. Deploy resources

### Setup Namespace with Minio Secret

In [8]:
!kubectl create namespace production

Error from server (AlreadyExists): namespaces "production" already exists


In [9]:
!kubectl apply -f ../../../k8s/tempo-pipeline-rbac.yaml -n production

serviceaccount/tempo-pipeline created
role.rbac.authorization.k8s.io/tempo-pipeline created
rolebinding.rbac.authorization.k8s.io/tempo-pipeline-rolebinding created


In [10]:
%%writefile minio-secret.yaml

apiVersion: v1
kind: Secret
metadata:
  name: minio-secret
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: minioadmin
  AWS_SECRET_ACCESS_KEY: minioadmin
  AWS_ENDPOINT_URL: http://minio.minio-system.svc.cluster.local:9000
  USE_SSL: "false"

Overwriting minio-secret.yaml


In [11]:
!kubectl apply -f minio-secret.yaml -n production

secret/minio-secret created


### Uploading artifacts

In [12]:
MINIO_IP=!kubectl get svc minio -n minio-system -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MINIO_IP=MINIO_IP[0]

In [13]:
%%writetemplate rclone.conf
[s3]
type = s3
provider = minio
env_auth = false
access_key_id = minioadmin
secret_access_key = minioadmin
endpoint = http://{MINIO_IP}:9000


In [14]:
import os
from tempo.conf import settings
settings.rclone_cfg = os.getcwd() + "/rclone.conf"

In [15]:
sklearn_model.upload()
xgboost_model.upload()
classifier.upload()

INFO:tempo:Uploading /home/clive/work/mlops/fork-tempo/docs/examples/intro_k8s/artifacts/sklearn to s3://tempo/basic/sklearn
INFO:tempo:Uploading /home/clive/work/mlops/fork-tempo/docs/examples/intro_k8s/artifacts/xgboost to s3://tempo/basic/xgboost
INFO:tempo:Uploading /home/clive/work/mlops/fork-tempo/docs/examples/intro_k8s/artifacts/classifier to s3://tempo/basic/pipeline


### Deploy

In [16]:
classifier.deploy()
classifier.wait_ready()

INFO:tempo:Deploying test-iris-sklearn
INFO:tempo:Deploying test-iris-xgboost
INFO:tempo:Deploying classifier


True

### Sending requests

Lastly, we can now send requests to our deployed pipeline.
For this, we will leverage the `remote()` method, which will interact without our deployed pipeline (as opposed to executing our pipeline's code locally).

In [17]:
classifier(payload=np.array([[1, 2, 3, 4]]))

(array([[0.00847207, 0.03168794, 0.95984   ]]), 'xgboost prediction')

In [18]:
classifier.remote(payload=np.array([[1, 2, 3, 4]]))

{'output0': array([[0.00847207, 0.03168794, 0.95984   ]]),
 'output1': 'xgboost prediction'}

In [19]:
classifier.remote(payload=np.array([[5.964,4.006,2.081,1.031]]))

{'output0': array([[1.00000000e+00, 1.02962312e-10, 1.64000397e-30]]),
 'output1': 'sklearn prediction'}

### Undeploy pipeline

In [24]:
classifier.undeploy()

INFO:tempo:Undeploying classifier
INFO:tempo:Undeploying test-iris-sklearn
INFO:tempo:Undeploying test-iris-xgboost
